In [2]:
import os
import numpy as np
import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Function to create dataset from loaded images and labels
def create_dataset_from_arrays(images, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Load saved dataset (same as before)
def load_dataset(save_dir, dataset_name):
    images_path = os.path.join(save_dir, f"{dataset_name}_images.npy")
    labels_path = os.path.join(save_dir, f"{dataset_name}_labels.npy")
    
    # Load the images and labels from the npy files
    images = np.load(images_path)
    labels = np.load(labels_path)
    
    print(f"{dataset_name} dataset loaded successfully.")
    return images, labels

# Load the dataset (adjust the path as needed)
images, labels = load_dataset('saved_datasets', 'dataset')

# Split the data (same as before)
from sklearn.model_selection import train_test_split
file_paths_train, file_paths_temp, labels_train, labels_temp = train_test_split(images, labels, test_size=0.4, stratify=labels)
file_paths_val, file_paths_test, labels_val, labels_test = train_test_split(file_paths_temp, labels_temp, test_size=0.5, stratify=labels_temp)

# Create datasets for evaluation
batch_size = 32
test_dataset = create_dataset_from_arrays(file_paths_test, labels_test, batch_size)

# Load the pre-trained model
model = load_model('best_model_13k_224.keras')

# Evaluate the model on the test dataset
print("Evaluating the model...")
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Get predictions
predictions = model.predict(test_dataset)
predicted_labels = np.argmax(predictions, axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(labels_test, predicted_labels, target_names=['Non Demented', 'Very mild Dementia']))

# Compute and display confusion matrix
cm = confusion_matrix(labels_test, predicted_labels)
print("Confusion Matrix:")
print(cm)





dataset dataset loaded successfully.
Evaluating the model...
163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9831 - loss: 0.0442
Test Loss: 0.0377
Test Accuracy: 0.9867
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step
Classification Report:
                    precision    recall  f1-score   support

      Non Demented       1.00      0.98      0.99      2600
Very mild Dementia       0.98      1.00      0.99      2600

          accuracy                           0.99      5200
         macro avg       0.99      0.99      0.99      5200
      weighted avg       0.99      0.99      0.99      5200

Confusion Matrix:
[[2543   57]
 [  12 2588]]
